In [ ]:
import pyodbc 
import pandas as pd
import numpy as np
from datetime import datetime

players = pd.read_csv("./People.csv", encoding='latin-1')
pitching = pd.read_csv("./Pitching.csv")
batting = pd.read_csv("./Pitching.csv")

print(players.columns)
print(players.shape)

print(pitching.columns)
print(pitching.shape)

print(batting.columns)
print(batting.shape)

#Server=localhost;Database=master;Trusted_Connection=True

Index(['ID', 'playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCity',
       'birthCountry', 'birthState', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'bbrefID',
       'finalGame', 'retroID'],
      dtype='object')
(21010, 25)
Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')
(51368, 30)
Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')
(51368, 30)


In [ ]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = ''

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [32]:
# Only need records from 1990 and greater
pitching = pitching[pitching['yearID'] >= 1990]
batting = batting[batting['yearID'] >= 1990]

players = players.dropna(subset=['birthYear', 'birthMonth', 'birthDay'], axis=0)
combined = pd.concat([batting['playerID'], pitching['playerID']], axis=0)
combined = combined.drop_duplicates()
print(combined)
print(combined.shape)

players_filtered = players[players['playerID'].isin(combined)]

print(players_filtered.shape)
print(players.shape)
print(pitching.shape)
print(batting.shape)

0        aardsda01
21        aasedo01
22        abadfe01
41       abbotan01
42       abbotco01
           ...    
51324    zobribe01
51347    zuberty01
51349    zumayjo01
51354    zuniggu01
51365     zychto01
Name: playerID, Length: 4925, dtype: object
(4925,)
(4925, 25)
(20593, 25)
(24074, 30)
(24074, 30)


In [ ]:
playerIDs = {}

for index, row in players_filtered.iterrows():
	systemPlayerID = row.playerID

	dateOfBirth = f"{int(row.birthYear)}-{int(row.birthMonth)}-{int(row.birthDay)}"
	print(dateOfBirth)
	print(row.nameLast)
	print(row.nameFirst)

	cursor.execute("SELECT PlayerID FROM MLB.Players WHERE LastName = ? AND DateOfBirth = ?", row.nameLast, dateOfBirth)
	player = cursor.fetchone()

	if (player is not None):
		print(player[0])
		playerIDs[systemPlayerID] = player[0]
	else:
		# Insert Player
		cursor.execute("""
									INSERT INTO MLB.Players 
								 	(FirstName,LastName,PositionCategory,Position,Status,DateOfBirth) 
								 	VALUES(?,?,?,?,?,?,?,?)""", 
									row.nameFirst, row.nameLast, row.PositionCategory, row.Position, row.Status, playerTeam, date)

print(playerIDs)



1981-12-27
Aardsma
David
1954-9-8
Aase
Don
1985-12-17
Abad
Fernando
4551
1995-9-20
Abbott
Cory
5866
1967-9-19
Abbott
Jim
1968-2-18
Abbott
Kyle
1967-9-15
Abbott
Paul
1995-9-26
Abreu
Albert
4972
1997-4-22
Abreu
Bryan
6508
1985-4-8
Abreu
Juan
1977-4-5
Abreu
Winston
1981-12-18
Accardo
Jeremy
1982-12-8
Aceves
Alfredo
1994-3-6
Acevedo
Domingo
4975
1977-12-18
Acevedo
Jose
1970-5-5
Acevedo
Juan
1988-8-27
Achter
A. J.
1958-9-24
Acker
Jim
1981-5-1
Acosta
Manny
1968-9-16
Acre
Mark
1991-8-4
Adam
Jason
4973
1986-8-19
Adams
Austin
1991-5-5
Adams
Austin
5676
1994-8-10
Adams
Chance
5612
1971-7-2
Adamson
Joel
1978-7-29
Adams
Mike
1973-3-6
Adams
Terry
1972-10-8
Adams
Willie
1988-2-25
Adcock
Nathan
1986-8-24
Adenhart
Nick


KeyboardInterrupt: 

In [27]:
for index, row in df.iterrows():
		playerID = None
		teamID = None
		
		cursor.execute("SELECT PlayerID FROM MLB.Players WHERE ExternalPlayerID = ?", row.PlayerID)
		player = cursor.fetchone()
		if (player is not None):
			playerID = player[0]

		cursor.execute("SELECT TeamID FROM MLB.Teams WHERE ExternalTeamID = ?", row.TeamID)
		team = cursor.fetchone()
		if (team is not None):
			teamID = team[0]

		if (playerID is None or teamID is None):
			continue

		cursor.execute("""
			INSERT INTO MLB.Batting
			(
					PlayerID, TeamID, Season, SeasonType, Games, Starts, AtBats, Runs, Hits, Singles,
					Doubles, Triples, HomeRuns, RunsBattedIn, BattingAverage, Outs, Strikeouts,
					Walks, HitByPitch, Steals, CaughtStealing, OBP, Slug, OBPPlus
			)
			VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
			""",
			playerID, teamID, row.Season, row.SeasonType, row.Games, row.Started, row.AtBats, row.Runs, row.Hits, row.Singles,
			row.Doubles, row.Triples, row.HomeRuns, row.RunsBattedIn, row.BattingAverage, row.Outs, row.Strikeouts,
			row.Walks, row.HitByPitch, row.StolenBases, row.CaughtStealing, row.OnBasePercentage, row.SluggingPercentage, row.OnBasePlusSlugging)
		
		# Only insert Pitchers
		if (row.PositionCategory == 'P'):
			cursor.execute("""
				INSERT INTO MLB.Pitching
				(
						PlayerID, TeamID, Season, SeasonType, Games, Starts, Wins, Losses, Saves, InningsPitched,
						ERA, Runs, Hits, HomeRuns, Strikeouts, StrikeoutsPerNineInnings, Walks, WalksPerNineInnings,
						PitchesThrown, WHIP, BattingAverageAgainst, OBP, Slug, OBPPlus, Shutouts
				)
				VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
				""",
				playerID, teamID, row.Season, row.SeasonType, row.Games, row.Started, row.Wins, row.Losses, row.Saves, row.InningsPitchedDecimal,
				row.EarnedRunAverage, row.PitchingRuns, row.PitchingHits, row.PitchingHomeRuns, row.PitchingStrikeouts, row.PitchingStrikeoutsPerNineInnings, row.PitchingWalks, row.PitchingWalksPerNineInnings,
				row.PitchesThrown, row.WalksHitsPerInningsPitched, row.PitchingBattingAverageAgainst, row.PitchingOnBasePercentage, row.PitchingSluggingPercentage, row.PitchingOnBasePlusSlugging, row.PitchingShutOuts)

cnxn.commit()
cursor.close()

"""
Index(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',
       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',
       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',
       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',
       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',
       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',
       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',
       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',
       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',
       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',
       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',
       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',
       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',
       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',
       'FantasyPointsDraftKings', 'WeightedOnBasePercentage',
       'PitchingCompleteGames', 'PitchingShutOuts', 'PitchingOnBasePercentage',
       'PitchingSluggingPercentage', 'PitchingOnBasePlusSlugging',
       'PitchingStrikeoutsPerNineInnings', 'PitchingWalksPerNineInnings',
       'PitchingWeightedOnBasePercentage', 'FantasyPointsBatting',
       'FantasyPointsPitching'],
"""

"\nIndex(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',\n       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',\n       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',\n       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',\n       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',\n       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',\n       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',\n       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',\n       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',\n       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',\n       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',\n       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',\n       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',\n       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',\n       'FantasyPointsDra